In [ ]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
from attacks import deepfool


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


img_size = 28
img_chan = 1
n_classes = 10


print('\nLoading MNIST')

dataset = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = dataset.load_data()
X_train = np.reshape(X_train, [-1, img_size, img_size, img_chan])
X_train = X_train.astype(np.float32) / 255
X_test = np.reshape(X_test, [-1, img_size, img_size, img_chan])
X_test = X_test.astype(np.float32) / 255

Categorical = tf.keras.utils.to_categorical
y_train = Categorical(y_train)
y_test = Categorical(y_test)

ind = np.random.permutation(X_train.shape[0])
X_train, y_train = X_train[ind], y_train[ind]

SPLIT = 0.1
n = int(X_train.shape[0] * (1-SPLIT))
X_valid = X_train[n:]
X_train = X_train[:n]
y_valid = y_train[n:]
y_train = y_train[:n]



def model(x, logits=False, training=False):
    with tf.variable_scope('conv0'):
        z = tf.layers.conv2d(x, filters=32, kernel_size=[3, 3],
                             padding='same', activation=tf.nn.relu)
        z = tf.layers.max_pooling2d(z, pool_size=[2, 2], strides=2)

    with tf.variable_scope('conv1'):
        z = tf.layers.conv2d(z, filters=64, kernel_size=[3, 3],
                             padding='same', activation=tf.nn.relu)
        z = tf.layers.max_pooling2d(z, pool_size=[2, 2], strides=2)

    with tf.variable_scope('flatten'):
        shape = z.get_shape().as_list()
        z = tf.reshape(z, [-1, np.prod(shape[1:])])

    with tf.variable_scope('mlp'):
        z = tf.layers.dense(z, units=128, activation=tf.nn.relu)
        z = tf.layers.dropout(z, rate=0.25, training=training)

    LOGITS = tf.layers.dense(z, units=10, name='logits')
    y = tf.nn.softmax(LOGITS, name='ybar')

    if logits:
        return y, LOGITS
    return y


class Dummy:
    pass


res = Dummy()


with tf.variable_scope('model'):
    res.x = tf.placeholder(tf.float32, (None, img_size, img_size, img_chan),
                           name='x')
    res.y = tf.placeholder(tf.float32, (None, n_classes), name='y')
    res.training = tf.placeholder_with_default(False, (), name='mode')

    res.ybar, logits = model(res.x, logits=True, training=res.training)

    with tf.variable_scope('acc'):
        count = tf.equal(tf.argmax(res.y, axis=1), tf.argmax(res.ybar, axis=1))
        res.acc = tf.reduce_mean(tf.cast(count, tf.float32), name='acc')

    with tf.variable_scope('loss'):
        xent = tf.nn.softmax_cross_entropy_with_logits(labels=res.y,
                                                       logits=logits)
        res.loss = tf.reduce_mean(xent, name='loss')

    with tf.variable_scope('train_op'):
        optimizer = tf.train.AdamOptimizer()
        res.train_op = optimizer.minimize(res.loss)

    res.saver = tf.train.Saver()

with tf.variable_scope('model', reuse=True):
    res.adv_epochs = tf.placeholder(tf.int32, (), name='adv_epochs')
    res.xadv = deepfool(model, res.x, epochs=res.adv_epochs)


graph = tf.InteractiveSession()
graph.run(tf.global_variables_initializer())
graph.run(tf.local_variables_initializer())


def evaluate(graph, res, X_data, y_data, batch_size=128):

    print('\nEvaluating')

    datasample = X_data.shape[0]
    databatch = int((datasample+batch_size-1) / batch_size)
    loss, acc = 0, 0

    for batch in range(databatch):
        print(' batch {0}/{1}'.format(batch + 1, databatch), end='\r')
        start = batch * batch_size
        end = min(datasample, start + batch_size)
        cnt = end - start
        batch_loss, batch_acc = graph.run(
            [res.loss, res.acc],
            feed_dict={res.x: X_data[start:end],
                       res.y: y_data[start:end]})
        loss += batch_loss * cnt
        acc += batch_acc * cnt
    loss /= datasample
    acc /= datasample

    print(' loss: {0:.4f} acc: {1:.4f}'.format(loss, acc))
    return loss, acc


def train(graph, res, X_data, y_data, X_valid=None, y_valid=None, epochs=1,
          load=False, shuffle=True, batch_size=128, name='model'):

    if load:
        if not hasattr(res, 'saver'):
            return print('\nError: cannot find saver op')
        print('\nLoading saved model')
        return res.saver.restore(graph, 'model/{}'.format(name))

    print('\nTrain model')
    datasample = X_data.shape[0]
    databatch = int((datasample+batch_size-1) / batch_size)
    for epoch in range(epochs):
        print('\nEpoch {0}/{1}'.format(epoch + 1, epochs))

        if shuffle:
            ind = np.arange(datasample)
            np.random.shuffle(ind)
            X_data = X_data[ind]
            y_data = y_data[ind]

        for batch in range(databatch):
            print(' batch {0}/{1}'.format(batch + 1, databatch), end='\r')
            start = batch * batch_size
            end = min(datasample, start + batch_size)
            graph.run(res.train_op, feed_dict={res.x: X_data[start:end],
                                              res.y: y_data[start:end],
                                              res.training: True})
        if X_valid is not None:
            evaluate(graph, res, X_valid, y_valid)

    if hasattr(res, 'saver'):
        print('\n Saving model')
        os.makedirs('model', exist_ok=True)
        res.saver.save(graph, 'model/{}'.format(name))


def predict(graph, res, X_data, batch_size=128):

    print('\nPredicting')
    n_classes = res.ybar.get_shape().as_list()[1]

    datasample = X_data.shape[0]
    databatch = int((datasample+batch_size-1) / batch_size)
    yval = np.empty((datasample, n_classes))

    for batch in range(databatch):
        print(' batch {0}/{1}'.format(batch + 1, databatch), end='\r')
        start = batch * batch_size
        end = min(datasample, start + batch_size)
        y_batch = graph.run(res.ybar, feed_dict={res.x: X_data[start:end]})
        yval[start:end] = y_batch
    print()
    return yval


def make_deepfool(graph, res, X_data, epochs=1, eps=0.01, batch_size=128):

    print('\nMaking adversarials via DeepFool')

    datasample = X_data.shape[0]
    databatch = int((datasample + batch_size - 1) / batch_size)
    X_adv = np.empty_like(X_data)

    for batch in range(databatch):
        print(' batch {0}/{1}'.format(batch + 1, databatch), end='\r')
        start = batch * batch_size
        end = min(datasample, start + batch_size)
        adv = graph.run(res.xadv, feed_dict={res.x: X_data[start:end],
                                            res.adv_epochs: epochs})
        X_adv[start:end] = adv
    print()

    return X_adv


print('\nTraining')

train(graph, res, X_train, y_train, X_valid, y_valid, load=False, epochs=5,
      name='mnist')

print('\nEvaluating on clean data')

evaluate(graph, res, X_test, y_test)

print('\nGenerating adversarial data')

X_adv = make_deepfool(graph, res, X_test, epochs=3)

print('\nEvaluating on adversarial data')

evaluate(graph, res, X_adv, y_test)


y1 = predict(graph, res, X_test)
y2 = predict(graph, res, X_adv)

z0 = np.argmax(y_test, axis=1)
z1 = np.argmax(y1, axis=1)
z2 = np.argmax(y2, axis=1)

print('\nPlotting results')
fig = plt.figure(figsize=(10, 2.2))
gs = gridspec.GridSpec(2, 10, wspace=0.05, hspace=0.05)

for i in range(10):
    ind, = np.where(np.all([z0 == i, z1 == i, z2 != i], axis=0))
    ind = np.random.choice(ind)
    xcur = [X_test[ind], X_adv[ind]]
    ycur = y2[ind]
    zcur = z2[ind]

    for j in range(2):
        img = np.squeeze(xcur[j])
        ans = fig.add_subplot(gs[j, i])
        ans.imshow(img, cmap='gray', interpolation='none')
        ans.set_xticks([])
        ans.set_yticks([])
    ans.set_xlabel('{0} ({1:.2f})'.format(zcur, ycur[zcur]), fontsize=12)

print('\nSaving figure')
gs.tight_layout(fig)
os.makedirs('img', exist_ok=True)
plt.savefig('img/deepfool_mnist.png')

Instructions for updating:
non-resource variables are not supported in the long term

Loading MNIST
11501568/11490434 [==============================] - 0s 0us/step
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/convolutional.py:575: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/pooling.py:600: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: UserWarning: `tf.layers.conv2d` is deprecated and will b

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Instructions for updating:
Use fn_output_signature instead
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


/usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:458: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  return f(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:458: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  return f(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:458: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return f(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:458: UserWarning: `tf.layers.dropout` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dropout` instead.
  return f(*args, **kwargs)



Training

Train model

Epoch 1/5

Evaluating
 loss: 0.0670 acc: 0.9797

Epoch 2/5

Evaluating
 loss: 0.0464 acc: 0.9862

Epoch 3/5
 batch 422/422
Evaluating
 loss: 0.0412 acc: 0.9877

Epoch 4/5
 batch 422/422
Evaluating
 loss: 0.0360 acc: 0.9890

Epoch 5/5
 batch 422/422
Evaluating
 loss: 0.0346 acc: 0.9902

 Saving model

Evaluating on clean data

Evaluating
 loss: 0.0300 acc: 0.9898

Generating adversarial data

Making adversarials via DeepFool


Evaluating on adversarial data

Evaluating
 loss: 1.6812 acc: 0.1285

Predicting
 batch 79/79

Predicting
 batch 79/79

Plotting results

Saving figure
